# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Откройте и изучите файл

In [1]:
#подключение библиотек
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

In [2]:
#подключение датасета
data = pd.read_csv('/datasets/users_behavior.csv')

In [3]:
#просмотр первых 5 строк
data.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


Целевым столбцом в датасете является **is_ultra**. Стоит задачи классификации. 1 присваивается клиентам с рекомендацией тарифа **Ultra**.

## Разбейте данные на выборки

Так как спрятанной тестовой выборки нет, данные раделим  соотношении 3:1:1(60-20-20). Размеры тестового и валидационного наборов равны(по 20%).

In [4]:
# Разделим исходные данные на обучающую, валидационную и тестовую выборки в пропорции 3:1:1.
# разделяем данные на 60(обучающая выборка) и 40 процентов(тестовая+валидационная)
data_train, data_valid = train_test_split(data, test_size=0.4, random_state=12345)

# разделение оставшейся части(40) пополам по 20 процентов
data_valid, data_test = train_test_split(data_valid, test_size=0.5, random_state=12345)

## Исследуйте модели

В иссследовании будем использовать следующие модели:

* Дерево решений
* Случайный лес
* Логистическая регрессия

In [5]:
#сохранение признаков и целей в отдельных переменных
#обучающая выборка
features_train = data_train.drop('is_ultra', axis=1)
target_train = data_train['is_ultra']

#валидационная выборка
features_valid = data_valid.drop('is_ultra', axis=1)
target_valid = data_valid['is_ultra']

#тестовая выборка
features_test = data_test.drop('is_ultra', axis=1)
target_test = data_test['is_ultra']

### Дерево решений

In [7]:
best_model_tree = None
best_result_tree = 0
best_depth = 0

#эксперимент с depth
for depth in (1,6):
    model_tree = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    
    #обучение модели
    model_tree.fit(features_train, target_train)
    result = model_tree.score(features_valid, target_valid)
    
    #нахождение лучшего результата
    if result > best_result_tree:
        best_result_tree = result
        best_model_tree = model_tree
        best_depth = depth

#вывод на экран лучшего результата        
print("Accuracy лучшей модели Решающего дерева: ", best_result_tree, "при глубине дерева: ", best_depth)

Accuracy лучшей модели Решающего дерева:  0.7838258164852255 при глубине дерева:  6


### Случайный лес

In [8]:
best_result_forest = 0
best_model_forest = None
best_est = 0

#эксперимент с количеством оценщиков
for est in range(1, 11):
    model_forest = RandomForestClassifier(random_state=12345, n_estimators=est)
    
    #обучение модели
    model_forest.fit(features_train, target_train)
    result = model_forest.score(features_valid, target_valid)
    
    #нахождение лучшего результата
    if result > best_result_forest:
        best_model_foret = model_forest
        best_result_forest = result
        best_est = est
        
#вывод на экран лучшего результата         
print("Accuracy лучшей модели Случайного леса: ", best_result_forest, "с количеством оценщиков: ", best_est)       

Accuracy лучшей модели Случайного леса:  0.7853810264385692 с количеством оценщиков:  10


### Логистическая регресcия

In [9]:
best_model_logic = None
best_result_logic = 0

model_logic = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=1000)
    
#обучение модели
model_logic.fit(features_train, target_train)
result = model_logic.score(features_valid, target_valid)
    
#нахождение лучшего результата
if result > best_result_logic:
    best_model_logic = model_logic
    best_result_logic = result
            
#вывод на экран лучшего результата         
print("Accuracy лучшей модели Логистической регрессии: ", best_result_logic)

Accuracy лучшей модели Логистической регрессии:  0.7107309486780715


**Вывод:**
Выбранные модели дали лучшие результаты:

* Accuracy лучшей модели Решающего дерева:  0.7838258164852255 при глубине дерева:  6
* Accuracy лучшей модели Случайного леса:  0.7853810264385692 с количеством оценщиков:  10
* Accuracy лучшей модели Логистической регрессии:  0.7107309486780715

Победила модель Случайного леса. Ее и будем проверять на тестовой выборке.

## Проверьте модель на тестовой выборке

In [10]:
model = RandomForestClassifier(random_state=12345, n_estimators=10)
model.fit(features_train, target_train)
predictions_test = model.predict(features_test)
result = accuracy_score(target_test, predictions_test)
print("Accuracy модели на тестовой выборке: ", result)

Accuracy модели на тестовой выборке:  0.7807153965785381


На тестовой выборке модель показала немного меньший результат, чем на валидационной (0.7807153965785381 против 0.7853810264385692), но все равно достойный.

## (бонус) Проверьте модели на адекватность

In [ ]:
data['is_ultra'].value_counts(normalize=True)

Доля большей части выборки равна 0.6935286, а у выбранной модели Accuracy 0.7807153965785381, отсюда можно сделать вывод, что модель можно использовать. 

# Общий вывод

* в исследовании использовались модели: **Дерево решений, Случайный лес, Логистическая регрессия.**


* наибольшую точность показала модель: **Случайный лес с Accuracy 0.7853810264385692 и количеством оценщиков: 10.**


* Модель Случайного леса была проверена на тестовой выборке.**На тестовой выборке модель показала немного меньший результат, чем на валидационной (0.7807153965785381 против 0.7853810264385692), но все равно достойный.**


* была проведена попытка оценки адекватности модели, которая привела к положительному решению. **Доля большей части выборки равна 0.6935286, а у выбранной модели Accuracy 0.7807153965785381, отсюда можно сделать вывод, что модель можно использовать.**